In [ ]:
from ultralytics import YOLOE
# from tqdm import tqdm
from icecream import ic
import os
import logging
import numpy as np
import cv2
import glob
# import fiftyone as fo
# import fiftyone.utils.yolo as fouy
# import supervision as sv
from pathlib import Path
from datetime import datetime
import pandas as pd
import shapely
from shapely.geometry import Polygon

In [ ]:
ORIGINAL_IMAGE_DIR='../original_images'

# dt = datetime.now()
# timestamp = f'{dt.year}{dt.month:02d}{dt.day:02d}-{dt.hour:02d}{dt.minute:02d}'
# FO_DATASET_NAME = f'after-postproc-{timestamp}'

YOLO_ANNOTATION_DIR='../yolo_annotations'
os.makedirs(YOLO_ANNOTATION_DIR, exist_ok=True)

In [ ]:
# Set logging level to WARNING to suppress INFO logs from ultralytics
# logging.getLogger('ultralytics').setLevel(logging.INFO)
# logger = logging.getLogger()
# logger.basicConfig(level=logging.DEBUG)

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

# Functions

In [ ]:
def contour_list_to_yolo_annotation_file(contour_list, width, height, yolo_annotation_path):
    """ 
    Function called by test_handle_masks.
    
    Arguments:
    
    Inputs
        contour_list: list of contours for detected objects returned by handle_masks()
        width: width of original image
        height: height of original image
        
    Output:
        yolo_annotation_path: filepath for a YOLO annotation text file to be written by this function
    """
    polygon_str = ''    
    for contour in contour_list:
        polygon_str += '0 ' # class 0
        for point in contour:
            x = point[0][0]
            y = point[0][1]
            normalized_x = x / width
            normalized_y = y / height
            polygon_str += f' {normalized_x:.4f} {normalized_y:.4f}'
        polygon_str += '\n'        
    with open(yolo_annotation_path, 'w') as f:
        f.write(polygon_str[:-1]) # write polygon_str after removing last '\n'

In [ ]:
def annotate_image(original_image_path, yolo_annotation_path, annotated_image_path):
    """ 
    Function used by test_handle_masks.
    Gets an image from original_image_path and uses data from yolo_annotation_path to
    overlay polygons on it. Resulting image is written to annotated_image_path.
    """
    img = cv2.imread(original_image_path)
    height, width, _ = img.shape
    
    with open(yolo_annotation_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        stringlist = line.strip().split()[1:] # class at start of line is removed
        numberlist = list(map(float, stringlist))
        points_xyn = np.array(numberlist).reshape(-1, 2) # normalized points
        points_pix = (points_xyn * np.array([width, height])).astype(int) 
        img = cv2.polylines(img, pts=[points_pix], color=(0, 255, 0), isClosed=True, thickness=1)
    cv2.imwrite(annotated_image_path, img)
    


In [ ]:
def test_handle_masks(original_image_path, yolo_labels_path):
    """ 
    Test code for the handle_masks function.
    Steps:
    1. Detects coconut palms in the image specified by original_image_path using YOLOE.
    2. handle_masks() uses the prediction results as input to remove artifacts and 
    detections which meet the sides or top of the image.
    The new masks data are returned as countour_list.
    3. contour_list_to_annotation_file() creates a YOLO format annotation text file 
    from the contour_list data.
    4. annotate_image() creates a new image with the new contours overlaid on the 
    original image and saves the image to a file. 
    """
        
    # original_image_path = '/home/aubrey/Desktop/inat-coco-jb/images/test_images/66897148.jpg'
    # yolo_labels_path = 'yolo_labels/668997148.txt'
    # original_image_path = '/home/aubrey/Desktop/inat-coco-jb/images/test_images/117236387.jpg'
    # yolo_annotation_path = 'mytest.txt'
    # annotated_image_path = f'mytest_{Path(original_image_path).stem}_annotated.jpg'
    
    # step 1
    model = YOLOE("yoloe-11l-seg.pt") 
    names = ["coconut palm tree"] 
    model.set_classes(names, model.get_text_pe(names))
    results = model.predict(source=original_image_path, conf=0.05, verbose=False)
    result = results[0]
    
    # step 2
    mask_list, contour_list = handle_masks(result)
    height, width = mask_list[0].shape
    ic(len(mask_list), len(contour_list))
    ic(mask_list[0].shape)

    # create image showing masks
    img = np.zeros_like(mask_list[0], dtype=np.uint8)
    for i, mask in enumerate(mask_list):
        img += (mask//255) * (255-(i*50))
        cv2.imwrite('mytest_masks.png', img)

    # create image showing contours
    img = np.zeros_like(mask_list[0], dtype=np.uint8)
    for i, contour in enumerate(contour_list):
        img += cv2.polylines(img, pts=[contour], color=(255-i*50), isClosed=True, thickness=1)
        cv2.imwrite('mytest_contours.png', img)
    
    # step 3    
    contour_list_to_yolo_annotation_file(contour_list, width, height, yolo_annotation_path)
    
    # step 4
    annotate_image(original_image_path, yolo_annotation_path, annotated_image_path)
    
# # Usage example:
# test_handle_masks(
#     original_image_path='/home/aubrey/Desktop/inat-coco-jb/images/test_images/66897148.jpg',
#     yolo_annotation_path='yolo_labels/668997148.txt'
# )

In [ ]:
def keep_largest_component_opencv(mask):
    """
    Keeps only the largest connected component in a binary mask using OpenCV.

    Args:
        mask (np.ndarray): A binary input mask (0s and 255s).

    Returns:
        np.ndarray: A new mask containing only the largest connected component (0s and 255s).
    """
    # Ensure mask is in the correct format (uint8 binary)
    if mask.dtype != np.uint8:
        mask = (mask * 255).astype(np.uint8)

    # Calculate contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return np.zeros_like(mask, dtype=np.uint8), None

    # Get the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a new blank mask and draw only the largest contour
    new_mask = np.zeros_like(mask, dtype=np.uint8)
    cv2.drawContours(new_mask, [largest_contour], -1, 255, cv2.FILLED)
    # ic('keep', largest_contour)
    return new_mask, largest_contour

# Example Usage:
# Assume 'yolo_mask' is your input binary mask (e.g., a NumPy array of 0s and 255s)
# yolo_mask = ... 
# result_mask = keep_largest_component_opencv(yolo_mask)


In [ ]:
def mask_touches_left_right_or_top_of_image(mask):
    first_column = mask[:, 0] # first column (left)
    last_column = mask[:, -1] # last column (right)
    first_row = mask[0] # first row (top)
    mask_touches_left = np.sum(first_column) > 0
    mask_touches_right = np.sum(last_column) > 0
    mask_touches_top = np.sum(first_row) > 0
    
    # The result of the following conditional is np.True_ or np.False
    # These values are not equvalant to python's True and False
    # Hopfully, the following fixes this problem.
    if (mask_touches_left or mask_touches_right or mask_touches_top):
        return True
    else: 
        return False

# # Usage example
# # The source image contains 2 mask, the first is OK, the second touches the right edge of the image.
# model = YOLOE("yoloe-11l-seg.pt") 
# names = ["coconut palm tree"] 
# model.set_classes(names, model.get_text_pe(names))
# source = '/home/aubrey/Desktop/inat-coco-jb/images/test_images/66897148.jpg'  
# results = model.predict(source=source, conf=0.05)
# for result in results:
#     for mask in result.masks:
#         # convert mask from tensor to binary image
#         mask = mask.data[0].cpu().numpy() * 255 
#         ic(mask_touches_left_right_or_top_of_image(mask))
        
# # ic| mask_touches_left_right_or_top_of_image(mask): False
# # ic| mask_touches_left_right_or_top_of_image(mask): True

In [ ]:
def handle_masks(result): 
    """ 
    Postprocesses detections to remove artifacts and objects touching the 
    sides or top of the image.
    
    Returns:
        mask_list: a list of numpy arrays; each array is a grayscale image 
        contour_list: a list of contours with format: 
            [[[x1,y1],[x2,y2],...,[xn,yn]]] where [x,y] defines a pixel location  
    """ 
    mask_list = []
    contour_list = []
    if result.masks is None:
        return mask_list, contour_list # return emapty lists
           
    ic('creating blank binary image with same shape as the first mask')
    mask = result.masks.data[0].cpu().numpy()
    binary_image = np.zeros_like(mask, dtype=np.uint8)
    ic(binary_image)
    
    for mask_num, mask in enumerate(result.masks):
        
        # convert mask from tensor to binary image
        mask = mask.data[0].cpu().numpy() * 255 

        # # save mask to image file
        # source_filename = os.path.basename(result.path)
        # mask_filename = f'{source_filename.split('.')[0]}-{mask_num}.jpg'
        # cv2.imwrite(mask_filename, mask)
        
        # remove artifacts
        mask, contour = keep_largest_component_opencv(mask)
        # ic('handle_masks', contour)

        # if this mask has pixels in the first column (left) or last column (right) or first row (top)
        # it is not added to the binary image   
        ic(mask_touches_left_right_or_top_of_image(mask))
           
        if not mask_touches_left_right_or_top_of_image(mask):
            # binary_image += mask
            mask_list.append(mask)
            contour_list.append(contour)
    
    # ic('saving binary_image to file')     
    # binary_image_filename = f"{source_filename.split('.')[0]}-mask.jpg"
    # if not cv2.imwrite(binary_image_filename, binary_image):
    #     ic(f'ERROR: Failed to write binary image to {binary_image_filename}') 
    
    return mask_list, contour_list  

# # Usage example:
# original_image_path = '/home/aubrey/Desktop/inat-coco-jb/images/test_images/117236387.jpg'
# model = YOLOE("yoloe-11l-seg.pt") 
# names = ["coconut palm tree"] 
# model.set_classes(names, model.get_text_pe(names))
# results = model.predict(source=original_image_path, conf=0.05, verbose=False)
# result = results[0]
# mask_list, contour_list = handle_masks(result) 

In [ ]:
def save_segmentation_results_to_yolo_format(
    results, 
    output_labels_dir="predicted_seg_labels",
    reject_objects_touching_image_sides_or_top=True,
    use_only_largest_contour_per_object=True):
    """ 
    Save segmentation results in YOLO segmentation format.
    """    
    os.makedirs(output_labels_dir, exist_ok=True)

    for i, result in enumerate(results):
    
        # Extract masks and class indices
        masks = result.masks.data if result.masks is not None else None
        classes = result.boxes.cls.cpu().numpy() if result.boxes is not None else None

        if masks is None or classes is None:
            continue
        
        ic(result.path)

        image_name = os.path.basename(result.path)
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(output_labels_dir, label_name)
        
        yolo_annotations = []

        # Loop through each instance with a detected mask
        for j, mask in enumerate(masks):
            class_index = int(classes[j])
                            
            # Format: class_id x1 y1 x2 y2 ... xn yn 
            mask_xyn = result.masks.xyn[j]              
            mask_xyn_list = mask_xyn.tolist()
            flat_mask_xyn_list = [coordinate for point in mask_xyn_list for coordinate in point] 
            
            if use_only_largest_contour_per_object:
                new_mask = get_largest_mask(
                    filename=os.path.basename(result.path),
                    mask=mask,
                    mask_num=j,
                    output_image_dir='annotated_images/predict')
                

            if reject_objects_touching_image_sides_or_top:
                # Check if any x or y coordinates are at the image sides or top
                xs = flat_mask_xyn_list[0::2]
                ys = flat_mask_xyn_list[1::2]
                if min(xs) < 0.01 or max(xs) > 0.99 or min(ys) < 0.01:
                    # ic(f"Rejecting object {j} in image {i} touching image sides or top")
                    continue
                     
            polyline_str = " ".join([f"{x:.4f}" for x in flat_mask_xyn_list])        
            annotation = f"{class_index} {polyline_str}\n"
            yolo_annotations.append(annotation)

        # Write annotations to the text file
        with open(label_path, "w") as f:
            f.writelines(yolo_annotations)
 
# # --- Example Usage ---
# # 1. Load a segmentation model (e.g., yolov8n-seg.pt)
# model = YOLO("yolov8n-seg.pt")

# # 2. Run inference to get results (ensure you use a segmentation model)
# # You can stream results from a directory
# results = model(source="your/image/directory", stream=True)

# # 3. Save results in the YOLO segmentation format
# save_segmentation_results_to_yolo_format(results)
# save_segmentation_results_to_yolo_format(
#     results, 
#     output_labels_dir="predicted_seg_labels",
#     reject_objects_touching_image_sides_or_top=True,
#     use_only_largest_contour_per_object=False):


In [ ]:
def write_yolo_label_txt(result):

        image_name = os.path.basename(result.path)
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(output_labels_dir, label_name)
        
        yolo_annotations = []

        # Loop through each instance with a detected mask
        for j, mask in enumerate(masks):
            class_index = int(classes[j])
                            
            # Format: class_id x1 y1 x2 y2 ... xn yn 
            mask_xyn = result.masks.xyn[j]              
            mask_xyn_list = mask_xyn.tolist()
            flat_mask_xyn_list = [coordinate for point in mask_xyn_list for coordinate in point] 
            
            if use_only_largest_contour_per_object:
                new_mask = get_largest_mask(
                    filename=os.path.basename(result.path),
                    mask=mask,
                    mask_num=j,
                    output_image_dir='annotated_images/predict')
                

            if reject_objects_touching_image_sides_or_top:
                # Check if any x or y coordinates are at the image sides or top
                xs = flat_mask_xyn_list[0::2]
                ys = flat_mask_xyn_list[1::2]
                if min(xs) < 0.01 or max(xs) > 0.99 or min(ys) < 0.01:
                    # ic(f"Rejecting object {j} in image {i} touching image sides or top")
                    continue
                     
            polyline_str = " ".join([f"{x:.4f}" for x in flat_mask_xyn_list])        
            annotation = f"{class_index} {polyline_str}\n"
            yolo_annotations.append(annotation)

        # Write annotations to the text file
        with open(label_path, "w") as f:
            f.writelines(yolo_annotations)


In [ ]:
def convert_seg_labels_to_polylines(label_path):
    """ 
    Convert YOLO segmentation labels to FiftyOne polylines.
    label_path : path to the YOLO segmentation label file
    Returns a list of FiftyOne Polyline objects.
    """
    polylines = []    
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        coords = list(map(float, parts[1:]))
        points = [[(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]]
        poly = fo.Polyline(points=points, closed=True, filled=True, label=class_id)
        polylines.append(poly)
    
    return polylines

# # --- Example Usage ---
# # 1. Convert a YOLO segmentation label file to FiftyOne polylines
# label_path = 'predicted_seg_labels/265378773.txt'
# polylines = convert_seg_labels_to_polylines(label_path)
# ic(polylines)

In [ ]:
def create_fo_dataset_with_polylines(fo_dataset_name, images_dir, anns_dir):
    """ 
    Create a FiftyOne dataset with polylines from predicted segmentation labels.
    """
    
    # # Create empty dataset
    # dataset = fo.Dataset(fo_dataset_name, persistent=True)
    

    # samples = []
    # for img_path in glob.glob(os.path.join(images_dir, "*")):
    #     if not os.path.isfile(img_path):
    #         continue

    #     sample = fo.Sample(filepath=img_path)

    #     # Derive annotation path from image filename (adapt as needed)
    #     stem = os.path.splitext(os.path.basename(img_path))[0]
    #     ann_path = os.path.join(anns_dir, stem + ".txt")
    #     if not os.path.exists(ann_path):
    #         samples.append(sample)
    #         continue

    #     # Attach polylines under field "polylines"
    #     polylines = convert_seg_labels_to_polylines(ann_path)
    #     sample["polylines"] = fo.Polylines(polylines=polylines)
    #     samples.append(sample)
 
    dataset = fo.Dataset.from_images_dir(
        images_dir=IMAGE_DIR,
        name=FO_DATASET_NAME,
        persistent=True,
    ) 
    fouy.add_yolo_labels(
        dataset,
        label_field="polygons",          # new field to hold labels
        labels_path=YOLO_ANNOTATION_DIR, # directory of .txt files
        classes=['coconut-tree'],
        label_type="polylines",          # IMPORTANT: load as polygons
    )
        
    # dataset.add_samples(samples)
    # try:
    #     session = fo.launch_app(dataset, auto=False)
    # except:
    #     session = fo.launch_app(dataset, auto=False)
        
# # Example usage:
# create_fo_dataset_with_polylines(
#     fo_dataset_name=FO_DATASET_NAME,
#     images_dir=IMAGE_DIR,
#     anns_dir=YOLO_ANNOTATION_DIR)


In [ ]:
def convert_yolo_xyn_to_wkt(xyn_coords):
    """
    Converts normalized YOLO polygon coordinates to a WKT POLYGON string.

    Args:
        xyn_coords (list or numpy array): A list of [x, y] points, normalized (0-1).

    Returns:
        str: The WKT representation of the polygon.
    """
    # if not xyn_coords:
    #     return None

    # Shapely Polygon expects a list of (x, y) tuples
    polygon_coords = [tuple(point) for point in xyn_coords]

    # Create a shapely Polygon object
    # Note: WKT format requires the first and last point to be the same to close the loop.
    # Shapely handles this automatically when creating the Polygon.
    try:
        polygon = Polygon(polygon_coords)
        # Convert the polygon object to a WKT string
        wkt_string = shapely.to_wkt(polygon)
        return wkt_string
    except Exception as e:
        print(f"Error creating Polygon: {e}")
        return None

# # --- Example Usage (assuming you have YOLO prediction results) ---

# # Mock function to simulate getting xyn results from YOLO
# def get_mock_yolo_results():
#     # Example normalized coordinates for a single polygon
#     # These values are between 0 and 1
#     normalized_polygon_points = [
#         [0.1, 0.2], [0.3, 0.1], [0.8, 0.4], [0.6, 0.9], [0.1, 0.8]
#     ]
#     # In a real scenario, this would come from:
#     # results = model.predict(source)
#     # first_mask_xyn = results[0].masks.xyn[0]
#     return normalized_polygon_points

# # Get the coordinates
# coords = get_mock_yolo_results()

# # Convert to WKT
# wkt_output = convert_yolo_xyn_to_wkt(coords)

# if wkt_output:
#     print(f"YOLO normalized coordinates: {coords}")
#     print(f"WKT Polygon: {wkt_output}")


# Tests

In [ ]:
def test_handle_masks(original_image_path, yolo_annotation_path):
    """ 
    Test code for the handle_masks function.
    Steps:
    1. Detects coconut palms in the image specified by original_image_path using YOLOE.
    2. handle_masks() uses the prediction results as input to remove artifacts and 
    detections which meet the sides or top of the image.
    The new masks data are returned as countour_list.
    3. contour_list_to_annotation_file() creates a YOLO format annotation text file 
    from the contour_list data.
    4. annotate_image() creates a new image with the new contours overlaid on the 
    original image and saves the image to a file. 
    """
        
    # original_image_path = '/home/aubrey/Desktop/inat-coco-jb/images/test_images/66897148.jpg'
    # yolo_labels_path = 'yolo_labels/668997148.txt'
    original_image_path = '/home/aubrey/Desktop/inat-coco-jb/images/test_images/117236387.jpg'
    yolo_annotation_path = 'mytest.txt'
    annotated_image_path = f'mytest_{Path(original_image_path).stem}_annotated.jpg'
    
    # step 1
    model = YOLOE("yoloe-11l-seg.pt") 
    names = ["coconut palm tree"] 
    model.set_classes(names, model.get_text_pe(names))
    results = model.predict(source=original_image_path, conf=0.05, verbose=False)
    result = results[0]
    
    # step 2
    mask_list, contour_list = handle_masks(result)
    height, width = mask_list[0].shape
    ic(len(mask_list), len(contour_list))
    ic(mask_list[0].shape)

    # create image showing masks
    img = np.zeros_like(mask_list[0], dtype=np.uint8)
    for i, mask in enumerate(mask_list):
        img += (mask//255) * (255-(i*50))
        cv2.imwrite('mytest_masks.png', img)

    # create image showing contours
    img = np.zeros_like(mask_list[0], dtype=np.uint8)
    for i, contour in enumerate(contour_list):
        img += cv2.polylines(img, pts=[contour], color=(255-i*50), isClosed=True, thickness=1)
        cv2.imwrite('mytest_contours.png', img)
    
    # step 3    
    contour_list_to_yolo_annotation_file(contour_list, width, height, yolo_annotation_path)
    
    # step 4
    annotate_image(original_image_path, yolo_annotation_path, annotated_image_path)
    
# # Usage example:
# test_handle_masks(
#     original_image_path='/home/aubrey/Desktop/inat-coco-jb/images/test_images/66897148.jpg',
#     yolo_annotation_path='yolo_labels/668997148.txt'
# )

# MAIN

In [ ]:
# Auto-annotate coconut palms in test_images and save results in YOLO incident segmentation format

# inputs: 
#   test_images/

# outputs: 
#   yoloe-11l-seg.pt
#   mobileclip_blt.ts
#   annotated_images/
#   predicted_seg_labels/

############################################
logging.info('    initializing YOLOE model')
############################################
# When first executed, this script will download the YOLOE-11L-Seg model weights (~90MB) and mobileclip_blt.ts (~300MB)
# These files are too large for pushing to GitHub so they should be added to .gitignore
ic.disable()
model = YOLOE("yoloe-11l-seg.pt") 
names = ["coconut palm tree"] 
model.set_classes(names, model.get_text_pe(names))

###########################################
logging.info('    detecting coconut palms')
###########################################
ic.enable()
data_list = []
image_paths = sorted(glob.iglob(f'{ORIGINAL_IMAGE_DIR}/*.jpg'))
for image_num, image_path in enumerate(image_paths):
    if image_num == 100:
        break
    results = model.predict(
        source=image_path, 
        imgsz=960,
        conf=0.01, 
        save=True, 
        project='annotated_images', 
        name='', 
        exist_ok=True, 
        verbose=False, 
        stream=False,
        save_crop=True
    )
    
    result = results[0]
    ic(image_num, '--------------------------------')
    
    if result.boxes is not None:  
        for  box_id, box in enumerate(result.boxes.xyxy):
            ic(box_id, '---')
            data_dict = {
                'image_id': Path(result.path).stem,
                'box_id': box_id,
                'conf': result.boxes.conf[box_id].item(),
                'polygon': convert_yolo_xyn_to_wkt(result.masks.xyn[box_id])
            }
            # ic(data_dict)
            # ic('before appending', data_list)
            # ic('appending', data_dict)
            data_list.append(data_dict)
            # ic('after_appending', data_list)
        
ic(data_list)  
df = pd.DataFrame(data_list)
df

    
    # if result.masks is None:
    #     ic('No masks')
    #     continue
    # mask_list, contour_list = handle_masks(result)
    # ic('main', contour_list)
    # ic(len(mask_list), len(contour_list))
 
######################################   
# create and save yolo annotation file
######################################  
    # if len(mask_list) > 0:
    #     height, width = mask_list[0].shape 
    # yolo_annotation_path = f'{YOLO_ANNOTATION_DIR}/{Path(result.path).stem}.txt'    
    # contour_list_to_yolo_annotation_file(
    #     contour_list = contour_list, 
    #     width=width, 
    #     height=height, 
    #     yolo_annotation_path=yolo_annotation_path
    # )
   
# ic('saving predicted segmentation labels')
# save_segmentation_results_to_yolo_format(
#     results, 
#     output_labels_dir="predicted_seg_labels",
#     reject_objects_touching_image_sides_or_top=True,
#     use_only_largest_contour_per_object=True)

# ############################################################
# logging.info('    creating fiftyone dataset with polylines')
# ############################################################
# create_fo_dataset_with_polylines(
#     fo_dataset_name=FO_DATASET_NAME,
#     images_dir=IMAGE_DIR,
#     anns_dir=YOLO_ANNOTATION_DIR)

# # ##########################################
# # logging.info('    launching FiftyOne app')
# # ##########################################
# # try:
# #     fo.launch_app(FO_DATASET_NAME, auto=False)
# # except:
# #     fo.launch_app(FO_DATASET_NAME, auto=False)
    
#############################
logging.info('    finished');
#############################

In [ ]:
df

In [ ]:
ic(result.masks.xyn[1])

In [ ]:
data_list

In [ ]:
result.boxes.xyxy

In [ ]:
results[0]

In [ ]:
for result in results:
    ic(result)